# ⚽ Kalman Filter Ball Chase

The red ball uses a Kalman filter to predict where the blue ball will be. The blue ball moves in a circular path while the Kalman filter learns its trajectory and guides the red ball to intercept it.

Adjust the measurement noise to see how the filter adapts to sensor uncertainty!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Circle
from IPython.display import HTML
from ipywidgets import interact, FloatSlider, Button, VBox, HBox, Output
import ipywidgets as widgets

%matplotlib inline

## Kalman Filter Implementation

In [ ]:
class KalmanFilter2D:
    """2D Kalman Filter for position and velocity tracking"""
    
    def __init__(self, process_noise=0.1, measurement_noise=40):
        # State: [x, y, vx, vy]
        self.state = np.array([350.0, 350.0, 0.0, 0.0])
        
        # State covariance matrix (4x4)
        self.P = np.array([
            [1000, 0, 0, 0],
            [0, 1000, 0, 0],
            [0, 0, 100, 0],
            [0, 0, 0, 100]
        ], dtype=float)
        
        # Process noise covariance
        self.Q = np.array([
            [process_noise, 0, 0, 0],
            [0, process_noise, 0, 0],
            [0, 0, process_noise * 10, 0],
            [0, 0, 0, process_noise * 10]
        ], dtype=float)
        
        # Measurement noise covariance
        self.R = np.array([
            [measurement_noise, 0],
            [0, measurement_noise]
        ], dtype=float)
        
        self.dt = 0.016  # ~60fps
        
        # State transition matrix F (4x4)
        self.F = np.array([
            [1, 0, self.dt, 0],
            [0, 1, 0, self.dt],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ], dtype=float)
        
        # Measurement matrix H (2x4) - we only measure position
        self.H = np.array([
            [1, 0, 0, 0],
            [0, 1, 0, 0]
        ], dtype=float)
    
    def predict(self):
        """Prediction phase: Project state and covariance forward"""
        # State prediction: x̂⁻(k) = F · x̂(k-1)
        self.state = self.F @ self.state
        
        # Covariance prediction: P⁻(k) = F·P(k-1)·Fᵀ + Q
        self.P = self.F @ self.P @ self.F.T + self.Q
    
    def update(self, measurement):
        """Update phase: Correct prediction with measurement"""
        # Innovation: y(k) = z(k) - H·x̂⁻(k)
        innovation = measurement - self.H @ self.state
        
        # Innovation covariance: S(k) = H·P⁻(k)·Hᵀ + R
        S = self.H @ self.P @ self.H.T + self.R
        
        # Kalman gain: K(k) = P⁻(k)·Hᵀ·S⁻¹(k)
        K = self.P @ self.H.T @ np.linalg.inv(S)
        
        # State update: x̂(k) = x̂⁻(k) + K(k)·y(k)
        self.state = self.state + K @ innovation
        
        # Covariance update: P(k) = (I - K(k)·H)·P⁻(k)
        I = np.eye(4)
        self.P = (I - K @ self.H) @ self.P
    
    def get_position(self):
        return self.state[0], self.state[1]
    
    def get_velocity(self):
        return self.state[2], self.state[3]
    
    def set_measurement_noise(self, noise):
        self.R = np.array([
            [noise, 0],
            [0, noise]
        ], dtype=float)
    
    def reset(self):
        self.state = np.array([350.0, 350.0, 0.0, 0.0])
        self.P = np.array([
            [1000, 0, 0, 0],
            [0, 1000, 0, 0],
            [0, 0, 100, 0],
            [0, 0, 0, 100]
        ], dtype=float)

## Ball Chase Simulation

In [ ]:
class BallChaseSimulation:
    """Simulation of red ball chasing blue ball using Kalman filter"""
    
    def __init__(self, center=(350, 350), radius=120, measurement_noise=40):
        self.center = np.array(center, dtype=float)
        self.radius = radius
        
        # Blue ball (target) - moves in circle
        self.blue_angle = 0
        self.blue_speed = 0.008  # radians per frame
        self.blue_radius = 12
        
        # Red ball (hunter)
        self.red_pos = np.array([self.center[0] - self.radius, self.center[1]], dtype=float)
        self.red_radius = 12
        self.red_speed = self.radius * self.blue_speed  # Match linear speed
        
        # Kalman filter
        self.kalman = KalmanFilter2D(process_noise=0.1, measurement_noise=measurement_noise)
        self.measurement_noise = measurement_noise
        
        # Tracking
        self.caught = False
        self.catch_distance = 25
        
        # Trails
        self.blue_trail = []
        self.red_trail = []
        self.pred_trail = []
        self.max_trail = 200
        
        # Current positions
        self.blue_pos = self.center + np.array([self.radius, 0])
        self.pred_pos = self.center + np.array([self.radius, 0])
    
    def update(self):
        """Update simulation one frame"""
        # Update blue ball position (circular motion)
        self.blue_angle += self.blue_speed
        self.blue_pos = self.center + self.radius * np.array([
            np.cos(self.blue_angle),
            np.sin(self.blue_angle)
        ])
        
        # Add measurement noise
        noise = np.random.randn(2) * np.sqrt(self.measurement_noise)
        measured_pos = self.blue_pos + noise
        
        # Kalman filter prediction and update
        self.kalman.predict()
        self.kalman.update(measured_pos)
        
        # Get Kalman estimate
        kalman_x, kalman_y = self.kalman.get_position()
        
        # Predict future position (several frames ahead)
        future_steps = 5
        future_angle = self.blue_angle + self.blue_speed * future_steps
        self.pred_pos = self.center + self.radius * np.array([
            np.cos(future_angle),
            np.sin(future_angle)
        ])
        
        # Red ball moves towards predicted position
        direction = self.pred_pos - self.red_pos
        distance = np.linalg.norm(direction)
        
        if distance > 0:
            direction = direction / distance
            self.red_pos += direction * self.red_speed
        
        # Check if caught
        catch_dist = np.linalg.norm(self.blue_pos - self.red_pos)
        self.caught = catch_dist < self.catch_distance
        
        # Update trails
        self.blue_trail.append(self.blue_pos.copy())
        self.red_trail.append(self.red_pos.copy())
        self.pred_trail.append(self.pred_pos.copy())
        
        if len(self.blue_trail) > self.max_trail:
            self.blue_trail.pop(0)
            self.red_trail.pop(0)
            self.pred_trail.pop(0)
        
        return catch_dist
    
    def reset(self):
        """Reset simulation to initial state"""
        self.blue_angle = 0
        self.red_pos = np.array([self.center[0] - self.radius, self.center[1]], dtype=float)
        self.caught = False
        self.blue_trail = []
        self.red_trail = []
        self.pred_trail = []
        self.kalman.reset()
    
    def set_measurement_noise(self, noise):
        """Update measurement noise parameter"""
        self.measurement_noise = noise
        self.kalman.set_measurement_noise(noise)
    
    def set_radius(self, radius):
        """Update circle radius"""
        self.radius = radius
        self.red_speed = self.radius * self.blue_speed
        self.reset()

## Interactive Animation

In [ ]:
# Create simulation
sim = BallChaseSimulation(center=(350, 350), radius=120, measurement_noise=40)

# Create figure
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xlim(0, 700)
ax.set_ylim(0, 700)
ax.set_aspect('equal')
ax.set_facecolor('#1a1a2e')
ax.axis('off')

# Draw circular path
circle = Circle(sim.center, sim.radius, fill=False, edgecolor='white', 
                linestyle='--', linewidth=2, alpha=0.3)
ax.add_patch(circle)

# Initialize plot elements
blue_ball = Circle(sim.blue_pos, sim.blue_radius, color='#3498db', zorder=10)
red_ball = Circle(sim.red_pos, sim.red_radius, color='#e74c3c', zorder=10)
pred_dot = Circle(sim.pred_pos, 6, color='#2ecc71', alpha=0.7, zorder=9)

ax.add_patch(blue_ball)
ax.add_patch(red_ball)
ax.add_patch(pred_dot)

blue_trail_line, = ax.plot([], [], color='#3498db', alpha=0.3, linewidth=1.5)
red_trail_line, = ax.plot([], [], color='#e74c3c', alpha=0.3, linewidth=1.5)
pred_line, = ax.plot([], [], color='#2ecc71', alpha=0.5, linewidth=1.5, linestyle='--')

status_text = ax.text(350, 650, '🎯 Chasing...', ha='center', va='top', 
                     fontsize=16, color='white', weight='bold')

# Animation state
running = [True]

def animate(frame):
    if not running[0]:
        return blue_ball, red_ball, pred_dot, blue_trail_line, red_trail_line, pred_line, status_text
    
    # Update simulation
    distance = sim.update()
    
    # Update ball positions
    blue_ball.center = sim.blue_pos
    red_ball.center = sim.red_pos
    pred_dot.center = sim.pred_pos
    
    # Update trails
    if len(sim.blue_trail) > 1:
        blue_trail = np.array(sim.blue_trail)
        blue_trail_line.set_data(blue_trail[:, 0], blue_trail[:, 1])
        
        red_trail = np.array(sim.red_trail)
        red_trail_line.set_data(red_trail[:, 0], red_trail[:, 1])
    
    # Update prediction line
    pred_line.set_data([sim.red_pos[0], sim.pred_pos[0]], 
                       [sim.red_pos[1], sim.pred_pos[1]])
    
    # Update status
    if sim.caught:
        status_text.set_text('🎉 Caught! ✨')
        status_text.set_color('#ff9800')
    else:
        status_text.set_text('🎯 Chasing...')
        status_text.set_color('white')
    
    return blue_ball, red_ball, pred_dot, blue_trail_line, red_trail_line, pred_line, status_text

# Create animation
anim = FuncAnimation(fig, animate, frames=1000, interval=16, blit=True, repeat=True)

# Control widgets
def on_reset(b):
    sim.reset()

def on_pause(b):
    running[0] = not running[0]
    b.description = '▶️ Resume' if not running[0] else '⏸️ Pause'

def on_noise_change(change):
    sim.set_measurement_noise(change['new'])

def on_radius_change(change):
    sim.set_radius(change['new'])

reset_btn = Button(description='🔄 Reset')
reset_btn.on_click(on_reset)

pause_btn = Button(description='⏸️ Pause')
pause_btn.on_click(on_pause)

noise_slider = FloatSlider(value=40, min=1, max=150, step=1, 
                           description='Noise (σ²):', continuous_update=False)
noise_slider.observe(on_noise_change, names='value')

radius_slider = FloatSlider(value=120, min=50, max=200, step=10,
                           description='Radius:', continuous_update=False)
radius_slider.observe(on_radius_change, names='value')

controls = VBox([
    HBox([reset_btn, pause_btn]),
    noise_slider,
    radius_slider
])

display(controls)
plt.tight_layout()
plt.show()

## 📐 Kalman Filter: Mathematical Foundation

### The Core Idea: Prediction + Correction

The Kalman filter works in two alternating phases:

- **PREDICT:** "Based on physics and past behavior, where should the ball be now?"
- **UPDATE:** "I measured it at position Z. My prediction was X. Let me blend these intelligently."

### Problem Formulation

**PROCESS MODEL (How the system evolves):**
$$\mathbf{x}(k) = \mathbf{F} \cdot \mathbf{x}(k-1) + \mathbf{w}(k)$$

**MEASUREMENT MODEL (What we observe):**
$$\mathbf{z}(k) = \mathbf{H} \cdot \mathbf{x}(k) + \mathbf{v}(k)$$

### State Vector

$$\mathbf{x}(k) = \begin{bmatrix} x_{pos} \\ y_{pos} \\ v_x \\ v_y \end{bmatrix}$$

### The Algorithm

**PREDICTION PHASE:**

State Prediction:
$$\hat{\mathbf{x}}^-(k) = \mathbf{F} \cdot \hat{\mathbf{x}}(k-1)$$

Covariance Prediction:
$$\mathbf{P}^-(k) = \mathbf{F} \cdot \mathbf{P}(k-1) \cdot \mathbf{F}^T + \mathbf{Q}$$

**UPDATE PHASE:**

Innovation:
$$\mathbf{y}(k) = \mathbf{z}(k) - \mathbf{H} \cdot \hat{\mathbf{x}}^-(k)$$

Innovation Covariance:
$$\mathbf{S}(k) = \mathbf{H} \cdot \mathbf{P}^-(k) \cdot \mathbf{H}^T + \mathbf{R}$$

Kalman Gain (THE MAGIC):
$$\mathbf{K}(k) = \mathbf{P}^-(k) \cdot \mathbf{H}^T \cdot \mathbf{S}^{-1}(k)$$

Updated State:
$$\hat{\mathbf{x}}(k) = \hat{\mathbf{x}}^-(k) + \mathbf{K}(k) \cdot \mathbf{y}(k)$$

Updated Covariance:
$$\mathbf{P}(k) = (\mathbf{I} - \mathbf{K}(k) \cdot \mathbf{H}) \cdot \mathbf{P}^-(k)$$

### Matrix Dimensions

For our 2D tracking problem:
- State vector **x**: 4×1 (x, y, vx, vy)
- Measurement vector **z**: 2×1 (x_measured, y_measured)
- State transition **F**: 4×4
- Measurement matrix **H**: 2×4
- State covariance **P**: 4×4
- Process noise **Q**: 4×4
- Measurement noise **R**: 2×2
- Kalman gain **K**: 4×2

### Why It Works

The Kalman gain automatically learns how much to trust measurements versus predictions:
- If measurements are noisy (high R): Trust prediction more (K → 0)
- If prediction is uncertain (high P): Trust measurement more (K → 1)

Under Gaussian noise assumptions, the Kalman filter is provably optimal—it minimizes mean squared error!